# ABS Quarterly Consumer Price Index 6401

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Set-up-some-constants-we-will-use-across-all-plots" data-toc-modified-id="Set-up-some-constants-we-will-use-across-all-plots-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Set-up some constants we will use across all plots</a></span></li><li><span><a href="#Headline-CPI-for-Australia" data-toc-modified-id="Headline-CPI-for-Australia-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Headline CPI for Australia</a></span></li><li><span><a href="#Analytical-annual-series" data-toc-modified-id="Analytical-annual-series-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Analytical annual series</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
from pathlib import Path
import calendar
import re

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# local imports
from python_env import python_env
from finalise_plot import finalise_plot
from abs_common import (get_constants,
                        get_ABS_meta_and_data,
                        get_identifier,
                        plot_growth2)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')

In [2]:
cat_num = '6401'
CACHE_DIR, CHART_DIR, source = get_constants(cat_num)

## Python environment

In [3]:
python_env()

--------------------------------------------------
System:     Darwin
Release:    20.1.0
Machine:    x86_64
Processor:  i386
RAM:        64GB
--------------------------------------------------
Python:     3.8.3
Psutil:     5.7.0
Pandas:     1.0.5
Numpy:      1.18.5
Matplotlib: 3.2.2
--------------------------------------------------


## Get data from ABS

In [4]:
url_template = ('https://www.abs.gov.au/statistics/economy'
                '/price-indexes-and-inflation/consumer-price-index-australia/'
                'MONTH-YEAR/All%20time%20series%20spreadsheets.zip')

meta, data = get_ABS_meta_and_data(url_template, CACHE_DIR)

We need to cache this file
File for sep-2020 of size 3.6 MB


## Plot

### Set-up some constants we will use across all plots

In [5]:
RECENT = data.index.max() - pd.DateOffset(years=3)
plot_times = (None, RECENT)
plot_frames = (data.copy(), data[data.index >= RECENT].copy())
plot_tags = ('full', 'recent')

### Headline CPI for Australia

In [6]:
table = '2'
series_type = 'Original'
title = 'All groups CPI ;  Australia'
quarterly_desc = f'Percentage Change from Previous Period ;  {title} ;'
annual_desc = f'Percentage Change from Corresponding Quarter of Previous Year ;  {title} ;'

identifiers = []
for description in (annual_desc, quarterly_desc):
    ident, units = get_identifier(meta, description, series_type, table)
    identifiers.append(ident)

# plot
for plot_from, plot_tag in zip(plot_times, plot_tags):
    plot_growth2(data[identifiers[0]], data[identifiers[1]], 
                  title, plot_from, plot_tag, CHART_DIR,
                  rfooter=f'{source} {table}',
                  lfooter=f'{series_type.capitalize()} series',
                  ppy=4 # four periods per year
                 )

### Analytical annual series

In [7]:
table = '8'
stem = 'Percentage Change from Corresponding Quarter of Previous Year ;  '
post = ' ;  Australia ;'
groups = [
    ['Tradables', 'Non-tradables'],
    ['Original', 'Seasonally Adjusted'],
    ['Goods', 'Services'],
    ['Trimmed Mean', 'Weighted Median'],
]

# manage long/ugly names
replacements = {
    'Goods': 'All groups, goods component',
    'Services': 'All groups, services component',
    'Original': 'All groups CPI',
    'Seasonally Adjusted': 'All groups CPI, seasonally adjusted',
}

for dataset in groups:
    idents = []
    for title in dataset:
        if title in replacements:
            title = replacements[title]
        description = f'{stem}{title}{post}'
        series_type = ('Seasonally Adjusted' if 'season' in title 
                            or 'Trimmed' in title
                            or 'Weighted' in title
                       else 'Original'
                      )
        ident, units = get_identifier(meta, description, series_type, table)
        idents.append(ident)
        
    for plot_frame, plot_tag in zip(plot_frames, plot_tags):
        pair = plot_frame[idents]
        pair = pair.dropna(how='all', axis=0)
        ax = pair.plot(lw=2, color=['blue', 'darkorange'])
        ax.legend(dataset, loc='best')
        finalise_plot(ax, f'CPI Australia: {", ".join(dataset)}', 
            units, plot_tag, CHART_DIR,
            rfooter=f'{source} {table}',
            lfooter='Per cent change from same quarter in previous year'
        )

## Finished

In [8]:
print('Finished')

Finished
